### Licença

Este Jupyter Notebook foi adaptado a partir do [tutorial oficial da Hopsworks](https://github.com/logicalclocks/hopsworks-tutorials/tree/master/advanced_tutorials/citibike), que está sob a [GNU Affero General Public License v3.0 (AGPLv3)](https://www.gnu.org/licenses/agpl-3.0.html). Em conformidade com esta licença, este trabalho e todas as obras derivadas devem também ser compartilhadas sob os mesmos termos.


# Previsão de Uso do Citibike

<small>01. Feature Backfill</small>

<center>
<img src='../../assets/m08_projeto_banner.png' alt='Penguins' width="600">
</center>

🗒️ O objetivo deste projeto é demonstrar criar uma solução real de *Machine Learning* para prever o número de usuários da Citi Bike em cada estação na cidade de Nova York. Especificamente, você irá criar uma *Feature Store* com o Hopsworks, e fazer o *deploy* de uma aplicação com o Streamlit. Este projeto está dividido nas seguintes partes:

- **Backfill de Features**: Como carregar, engenhar e criar grupos de features.
- **Pipeline de Features**: Como analisar novos dados e inseri-los nos grupos de features.
- **Pipeline de Treinamento**: Como construir uma visualização de features, divisão do conjunto de dados de treinamento, treinar um modelo e salvá-lo no Registro de Modelos.
- **Pipeline de Inferência**: Como recuperar um modelo treinado do registro de modelos e usá-lo para inferência em lote.
- **Implantar um aplicativo Streamlit**.

# Previsão de Uso da Citibike

Este é um exemplo avançado do uso do Hopsworks Feature Store, com a tarefa de prever o número de usuários da Citibike em cada estação na cidade de Nova York. O Feature Store é uma parte essencial da infraestrutura de IA que ajuda as organizações a trazer dados empresariais modernos para sistemas de ML analíticos e operacionais. É a maneira mais simples e poderosa de levar seus modelos para a produção, de qualquer lugar para qualquer lugar. Você carregará os dados iniciais no feature store, criará dois grupos de features a partir dos quais faremos uma visualização de features e um conjunto de dados de treinamento, e treinará um modelo para prever a quantidade de usuários. Além disso, você projetará um pipeline de geração de dados e inserção no Feature Store, que será executado periodicamente usando ações do GitHub. Um aplicativo Streamlit será criado para que você possa experimentar seu modelo de maneira interativa.

Este é um caso de uso em lote, que oferecerá uma visão de alto nível sobre como usar nossas APIs Python e a interface do usuário para navegar pelos grupos de features.


In [1]:
# Carregar variáveis de ambiente
from dotenv import load_dotenv
import os
load_dotenv()

# Bibliotecas padrão e de data/hora
from datetime import timedelta, datetime

# Bibliotecas de análise de dados
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar

# Bibliotecas de visualização
import plotly.express as px

# Módulos específicos do projeto
import sigmoidal as sig

# Configurações gerais
import warnings
warnings.filterwarnings("ignore")  # Ignorar avisos


In [2]:
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
HOPSWORKS_PROJECT_NAME = 'sigmoidal'

In [3]:
df_raw = sig.get_citibike_data("01/2023", "02/2023")

_____ Processando 01/2023... _____
https://s3.amazonaws.com/tripdata/202301-citibike-tripdata.csv.zip
Download iniciado...⏳
Download concluído!👌
Recuperando DataFrame do arquivo csv...💿
--------------------------------
        date station_id  users_count
0 2023-01-01    2733.03            4
1 2023-01-01    2782.02            2
2 2023-01-01    2821.05            4
3 2023-01-01    2832.03            6
4 2023-01-01    2872.02            1
         date station_id  users_count
6  2023-01-01    2932.03            3
7  2023-01-01    2951.05            2
19 2023-01-01    3117.05           10
20 2023-01-01    3125.09           11
25 2023-01-01    3166.03            3
_____ Processando 02/2023... _____
https://s3.amazonaws.com/tripdata/202302-citibike-tripdata.csv.zip
Download iniciado...⏳
Download concluído!👌
Recuperando DataFrame do arquivo csv...💿
--------------------------------
        date station_id  users_count
0 2023-02-01    2733.03            1
1 2023-02-01    2821.05            1
2

In [4]:
df_raw

,date,station_id,users_count
0,2023-02-01,2832.03,1
1,2023-02-01,2912.08,5
2,2023-02-01,2932.03,6
3,2023-02-01,3011.03,12
4,2023-02-01,3038.08,4
...,...,...,...
121602,2023-01-31,8226.06,8
121603,2023-01-31,8226.07,5
121604,2023-01-31,8336.02,6
121605,2023-01-31,8511.08,1
